In [ ]:
%config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(16,8))
rc('font',size=12)

from scipy.signal import medfilt
from scipy.interpolate import interp1d
from sklearn.cluster import DBSCAN

from qubicpack.qubicfp import qubicfp
from qubic import fibtools as ft

from importlib import reload
import healpy as hp

import time_domain_tools as tdt

## Looking at all TES in order to asses the quality and see common features


In [ ]:
mydatadir = '/Users/hamilton/Qubic/Calib-TD/'
thedate = '2022-04-16'
thedata = '2022-04-16_12.37.59__ScanMap_Speed_VE14_FastNoMod'
FreqSrc = 140.

# mydatadir = '/Users/hamilton/Qubic/Calib-TD/'
# thedate = '2022-04-14'
# thedata = '2022-04-14_13.17.33__ScanMap_Speed_VE14_FastNoMod'
# FreqSrc = 170.


filename = mydatadir + '/' + thedate + '/' + thedata

### Read data
a = qubicfp()
a.read_qubicstudio_dataset(filename)

tt, alltod = a.tod()

az = a.azimuth()
el = a.elevation()
thk = a.timeaxis(datatype='hk')

del(a)

### We remove tt[0]
tinit = tt[0]
tt -= tinit
thk -= tinit


### Scanning related information
we first apply the `identify_scans()` function to get relevant information from the scanning. It provides:
- scantype_hk (1,-1 for azmuth directions, 0 for extreme of the scans -> to be discarded)
- scantype, az, el sampled on the same time axis as the data

In [ ]:
# Identify scan types and numbers
%matplotlib inline
rc('figure',figsize=(16,8))

reload(tdt)
scantype_hk, azt, elt, scantype = tdt.identify_scans(thk, az, el, tt=tt, doplot=True, thr_speedmin=0.1)

Let's first plot all TES in order to have an idea of the abvious issues.

We clearly see the saturated ones here: they reach the maximum and minimum range in the plot below. SO we need to discard all TES that reach this value at a time. In a further analysis we may only cut the regions where saturation occurs. Here we decide to be a bit brutal and just remove any TES that reaches saturation, even for one time sample

In [ ]:
rc('figure',figsize=(16,8))
rc('font',size=12)

tst = 1000
dt = 3000

tes_list = np.arange(256)


ok =(tt > tst) & (tt < (tst+dt))

for i in range(len(tes_list)):
    plot(tt[ok], -alltod[i,ok], label='TES {}'.format(i))

In [ ]:
maxis = np.max(np.abs(alltod), axis=1)
print('Saturation value: {}'.format(np.max(maxis)))

frac = np.zeros(256)
for i in range(256):
    frac[i] = (np.abs(alltod[i,:])==np.max(maxis)).sum() / len(tt) * 100
a=hist(frac, range=[0,100], bins=1000)  
xlabel('Fraction of saturated time samples [%]')
ylabel('Number of TES')
xscale('log')
yscale('log')

oktes = (maxis < np.max(maxis))
print('Number of never-saturated TES: {}'.format(oks.sum()))

figure()
for i in range(len(tes_list)):
    if oks[i]:
        plot(tt[ok], -alltod[i,ok], label='TES {}'.format(i))

We need to reduce this as much as we can... Most of this is due to the jumps. Can we prevent that with QUBIC Studio, or at a deeper level ? I don't remember seeing this with data taken in Paris...

Anyway, let's continue with those "good TES"...

In [ ]:
# Covariance matrix of all TES
mycovmat = np.cov(alltod[oktes,:])

def cov2corr(mat):
    """
    Converts a Covariance Matrix in a Correlation Matrix
    """
    newmat = np.empty_like(mat)
    ll, cc = np.shape(mat)
    for i in range(ll):
        for j in range(cc):
            newmat[i, j] = mat[i, j] / np.sqrt(mat[i, i] * mat[j, j])
    return newmat

mycorrmat = cov2corr(mycovmat)

subplot(1,2,1)
imshow(np.log10(mycovmat))
colorbar()
subplot(1,2,2)
imshow(np.log10(mycorrmat))
colorbar()


In [ ]:
rc('figure',figsize=(20,12))
for j in range(8):
    figure()
    title('TES {} to {}'.format(j*32+1, (j+1)*32))
    for i in range(32):
        if oks[j*32+i]:
            plot(tt[ok], -alltod[j*32+i,ok], label='TES {}'.format(j*32+i+1))  
    legend(fontsize=8)
    show()

There a re many things to remark here, each will require a specific code for dealing with it:
- Jumps are a huge problem as said before (not in this reduced TES sample however)
- At each end of scan, we perform a FLL reset that produces some very noisy data. This part of the data will be flagged as bad and should actually be replaced by a constrained noise realization (set from the data before and after).

# Jumps detection
Haar wavelets (inspired form Camille Perbost PhD)

In [ ]:
%matplotlib notebook
# %matplotlib inline
%config InlineBackend.figure_format='retina'
rc('figure',figsize=(15,8))

reload(tdt)

# numTES = 163
# numTES = 134
# numTES = 33
numTES = 102

tmin = 0
tmax = np.max(tt)
ok = (tt>=tmin) & (tt <= tmax)

idx = np.arange(len(alltod[numTES-1,:]))
# ok = (idx > 1.18e6) & (idx < 1.25e6+10000)
# ok = (idx > 0.3e6) & (idx < 0.5e6)
# ok = (idx > (1.5e6+298000)) & (idx < (1.5e6+306000))
# ok = (idx > (1.5e6)) & (idx < (2.e6))


ddinit = -alltod[numTES-1,:][ok]
mytt = tt[ok]
flags = np.zeros(len(ddinit), dtype=int)    ### At first all time samples are considered OK


### Flag-out the regions at the azimuth-edge of the scans
flags[scantype[ok]==0] = 1e7   
dd = tdt.fill_bad_regions(ddinit, flags, fill_baselines='equalize')

figure()
idx = np.arange(len(dd))
subplot(2,1,1)
plot(idx, ddinit, label='Initial Data')
plot(idx[flags>0], ddinit[flags>0], 'g.', label='Flagged bad (scanning)')
legend()
xlabel('Samples')
ylabel('ADU')
title('Raw TOD')
subplot(2,1,2)
plot(idx, dd, label='Data')
plot(idx[flags>0], dd[flags>0], 'g.', label='Flagged bad')
xlabel('Samples')
ylabel('ADU')
title('Corrected for extreme scanning regions')
legend()
tight_layout()


newdd, flags_jumps = tdt.jumps_correction(dd, threshold=15, size_haar=51, doplot=False, verbose=False, method='continuity_lin')
flags += flags_jumps


figure()
idx = np.arange(len(dd))
subplot(2,1,1)
plot(idx, dd, label='Data')
plot(idx[flags>0], dd[flags>0], 'g.', label='Flagged bad')
plot(idx[flags_jumps>0], dd[flags_jumps>0], 'r.', label='Flagged bad for jumps')
legend()
xlabel('Samples')
ylabel('ADU')
title('Raw TOD')
subplot(2,1,2)
plot(idx, newdd, label='Data')
plot(idx[flags>0], newdd[flags>0], 'g.', label='Flagged bad')
plot(idx[flags_jumps>0], newdd[flags_jumps>0], 'r.', label='Flagged bad for jumps')
xlabel('Samples')
ylabel('ADU')
title('Corrected for jumps')
legend()
tight_layout()

In [ ]:
# Covariance matrix of all TES
mycovmat = np.cov(alltod)

def cov2corr(mat):
    """
    Converts a Covariance Matrix in a Correlation Matrix
    """
    newmat = np.empty_like(mat)
    ll, cc = np.shape(mat)
    for i in range(ll):
        for j in range(cc):
            newmat[i, j] = mat[i, j] / np.sqrt(mat[i, i] * mat[j, j])
    return newmat

mycorrmat = cov2corr(mycovmat)

subplot(1,2,1)
imshow(np.log10(mycovmat))
colorbar()
subplot(1,2,2)
imshow(np.log10(mycorrmat))
colorbar()


In [ ]:
### Now we need to discard bad TES, roughly intercalibrate TESs... Tricky...
allmedians = np.median(alltod, axis=1)
allstd = np.std(alltod, axis=1)

In [ ]:
mm, ss = ft.meancut(allmedians, 3)
okmed = np.abs(allmedians) < ss/5
okstd = allstd < 0.15e6

ok = okmed & okstd

np.sum(ok)

subplot(1,3,1)
plot(allmedians, allstd, 'ko', label='All')
plot(allmedians[ok], allstd[ok], 'ro', label='Kept')
xlabel('Median of TOD')
ylabel('RMS')
legend()

subplot(1,3,2)
a=hist(allmedians, range=[-5e6, 5e6], bins=101, color='k', alpha=0.3, label='All')
a=hist(allmedians[ok], range=[-5e6, 5e6], bins=101, color='r', alpha =0.3, label='Kept')
xlabel('Median of TOD')
legend()

subplot(1,3,3)
a=hist(allstd, range=[0, 3e6], bins=101, color='k', alpha=0.3, label='All')
a=hist(allstd[ok], range=[0, 3e6], bins=101, color='r', alpha =0.3, label='Kept')
xlabel('STD of TOD')
legend()

todsok = alltod[ok,:]
print('Kept: {} TES'.format(np.sum(ok)))
print(np.shape(todsok))

In [ ]:
todsok = ((todsok.T - allmedians[ok])).T
med_tod = np.median((todsok.T/allstd[ok]**2), 1).T

In [ ]:
import scipy
from scipy import ndimage
med_tod = scipy.ndimage.median_filter(med_tod, 10)